# Iterative Methods to Solve System of Linear Equations

## Jacobi Method

In [1]:
import numpy as np
from scipy.sparse import *
from scipy.linalg import norm

# Sauer's Sparse matrix set up translation to Python
def sparseSetup(n):
    e = np.ones(n)
    n2 = n/2
    a = dia_matrix(([-e, 3*e, -e], [-1, 0, 1]), shape=(n, n)).tocsr()
    b = np.ones(n)
    b[0] = 2; b[n-1] = 2
    return a, b

# Calculate the backward error
def backwardError(b, ax, guessVec):
    axa = ax.dot(guessVec)
    tonorm = b - axa
    abstonorm = np.absolute(tonorm)
    backwardErr = abstonorm.max()
    return backwardErr

# Function to solve a system of linear equations by Jacobi methods
def jacobi(a, b, tol):
    n = len(b)
    d = [a[i, i] for i in range(n)]
    r = a - np.diag(d)
    x = np.zeros((n, 1))
    p = np.ones((n, 1))
    bb = np.reshape(b, (n, 1))
    dd = np.reshape(d, (n, 1))
    ite = 0
    diff = tol * 2

    while diff > tol:
        x = (bb - np.dot(r, x)) / dd
        diff = norm(x - p, np.inf)
        ite += 1

    print ("Step needed for Jacobi method:", ite)
    bshape = b.reshape((n, 1))
    adense = a.todense()
    err = backwardError(bshape, adense, x)
    print ("Backward Error is", err)

    return p.T

Use the Jacobi Method to solve the sparse system within six correct decimal places (forward error in the infinity norm) for $n = 100$ and $n = 100000$. The correct solution is $[1,...,1]$. Report the number of steps needed and the backward error.

Given that $100000x100000$ is too computationally expensive to do. I will leave them out.

In [2]:
# For n = 100
tolerance = 0.00000025
a100,  b100 = sparseSetup(100)
print(jacobi(a100, b100, tolerance))

# For n = 10000
a10000, b10000 = sparseSetup(10000)
print (jacobi(a10000, b10000, tolerance))

Step needed for Jacobi method: 38
Backward Error is 2.0348810692016883e-07
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]]
Step needed for Jacobi method: 38
Backward Error is 2.0348810692016883e-07
[[1. 1. 1. ... 1. 1. 1.]]


## Gauss-Seidel Method and SOR

Rewrite the Program 2.2. to carry Gauss-Seidel iteration. Solve the problem in Example 2.24 (page 110) to check your work.

In [3]:
# Sparse matrix set up for the new problem
def sparsesetup(n):
    n2 = n / 2
    e = np.ones(n)
    a = dia_matrix(([-e, 3*e, -e], [-1, 0, 1]), shape=(n, n)).tocsr()
    c = coo_matrix((e / 2, (range(n), range(n - 1, -1, -1))), shape=(n, n)).tocsr()
    a = a+c
    a[n2, n2 - 1] = -1;
    a[n2 - 1, n2] = -1;
    b = np.ones(n)
    b[0] = 2.5
    b[5] = 2.5
    b[1] = 1.5
    b[4] = 1.5
    return a, b


def gaussseidel(a, b, tol):
    n = len(b)
    d = [a[i, i] for i in range(n)] 
    r = a - np.diag(d) 
    x = np.zeros((n, 1))
    p = np.ones((n, 1))
    bb = np.reshape(b, (n, 1))
    dd = np.reshape(d, (n, 1))
    ite = 0
    diff = 1
    adense = a.todense()
    lowertri = tril(r).toarray()
    uppertri = triu(r).toarray()

    while diff > tol:
        cont = np.dot(uppertri, x)

        for j in range(0, n):
            x[j] = (bb[j] - cont[j] - np.dot(lowertri[j, :], x)) / dd[j]

        diff = norm(x - p,  np.inf)
        ite += 1

    print ("Step needed for Gauss-Seidel method:", ite)
    bshape = b.reshape((n, 1))
    err = backwardError(bshape, adense, x)
    print ("Backward Error is", err)
    return x

In [5]:
# Using the same tolerance from the previous problem.
a, b = sparsesetup(6)
print (gaussseidel(a, b, tolerance))

Step needed for Gauss-Seidel method: 17
Backward Error is 1.7237836891226266e-07
[[1.0000001 ]
 [1.00000011]
 [1.00000006]
 [1.00000001]
 [0.99999997]
 [0.99999997]]


The solution for this problem is supposed to be $x = [1, 1, 1, 1, 1, 1]$ which is close enough.